In [22]:
import os
import re
import numpy as np
import pandas as pd
import json
from pprint import pprint
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool, cpu_count

### Determining food names and tables

In [29]:
root = "../raw/www.valori-alimenti.com"
foods = [food for food in os.listdir(root) if food[0] != "."]

### Merging csv files

In [30]:
def parse_food(food):
    root = "../raw/www.valori-alimenti.com"
    file_path = "{path}/{name}/0.csv".format(path=root, name=food)
    df = pd.read_csv(file_path, index_col=0)
    if df.empty:
        return None
    try:
        for index, value in df.iterrows():
            if pd.isna(value).any():
                df = df.drop(index)
        for index, (unit, value) in df.iterrows():
            if isinstance(value, str) and "tr" in value:
                value = 0.05
            if unit not in ["mg", "mcg", "g", "kcal"]:
                df = df.iloc[[0] + list(range(2, len(df.columns)))]
                continue
            if unit in ["mg", "mcg"]:
                if unit == "mg":
                    df.loc[index] = float(value) * 1e-3
                elif unit == "mcg":
                    df.loc[index] = float(value) * 1e-6
                unit = "g"
            df = df.rename(
                index={
                    index:
                    "{index} | {unit}".format(index=index, unit=unit).lower()
                    .strip().replace("  ", " ")
                })
    except Exception as e:
        pprint(food)
        pprint(df)
        raise e
    df = df.drop(columns=["1"])
    df = df.transpose()
    with open("{path}/{name}/metadata.json".format(path=root, name=food),
              "r") as f:
        df["category"] = json.load(f)["category"]
    df.index = [food]
    df.index.name = "name"
    del df.columns.name
    return df

In [31]:
from IPython.display import display
with Pool(cpu_count()) as p:
    valori_alimenti = pd.concat(list(tqdm(p.imap(parse_food, foods), total=len(foods))))
valori_alimenti.index.name = "name"
valori_alimenti.to_csv("../csv/valori_nutrizionali.csv")

In [32]:
valori_alimenti

,"acidi grassi, monoinsaturi | g","acidi grassi, polinsaturi | g","acidi grassi, saturi | g",acido ascorbico (vit. c) | g,acido folico (vit. b9 o m o folacina) | g,acido pantotenico (vit. b5) | g,alpha-tocoferolo (vit. e) | g,amido | g,beta-sistosterolo | g,betaina | g,...,sodio | g,stigmasterolo | g,teobromina | g,tiamina (vit. b1) | g,tocoferolo beta | g,tocoferolo delta | g,tocoferolo gamma | g,"vitamina b-12, aggiunta | g","vitamina e, aggiunta | g",zinco | g
name,,,,,,,,,,,,,,,,,,,,,
pasta sfoglia,NaN,NaN,NaN,NaN,NaN,NaN,0.00055,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0000,NaN
nestle - buitoni risotto di mare buitoni,NaN,NaN,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loacker classic chocolat dark noir 118 grammi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"croissant, al burro",NaN,NaN,NaN,NaN,NaN,NaN,0.00084,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0000,NaN
"lombo di maiale, magro, brasato",NaN,NaN,NaN,NaN,NaN,NaN,0.00021,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0000,NaN
muffin inglese preparati con farina di frumento,NaN,NaN,NaN,NaN,NaN,NaN,0.00045,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0000,NaN
loacker chip choc dark noir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
caviale,NaN,NaN,NaN,NaN,NaN,NaN,0.00189,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0000,NaN
barilla - mulino bianco pan di casa - filone grano tenero,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
np.mean(np.mean(pd.isna(valori_alimenti)))

0.8251546736525277